In [88]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import open3d as o3d

In [89]:
tool_2 = o3d.io.read_triangle_mesh("/calibration/objects/tools/tool_2.stl")

In [90]:
o3d.visualization.draw_geometries([tool_2])

# 1. Pointcloud postprocessing

### Outlier rejection

In [116]:
ply_file_path = "/calibration/data/scanner/stitched_pc.ply"  # Replace with your .ply file path
pcd = o3d.io.read_point_cloud(ply_file_path)
# Apply a transformation if needed (e.g., rotating or translating the point cloud)
# For example, a 90-degree rotation around the x-axis can be done as follows:
rotation_matrix = pcd.get_rotation_matrix_from_xyz((np.pi / 2, np.pi, 0))
pcd.rotate(rotation_matrix, center=(0, 0, 0))

o3d.visualization.draw_geometries([pcd])

In [123]:
def display_inlier_outlier(cloud, ind):
    
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [126]:
cleaned_pcd, ind = pcd.remove_statistical_outlier(
    nb_neighbors=20,
    std_ratio=1.5
)

display_inlier_outlier(pcd, ind)

Showing outliers (red) and inliers (gray): 


### ROI filtering

In [232]:
TOOLS_MIN_LENGTH = 0.02
TOOLS_MAX_LENGTH = 0.03
# OPTIONAL
CLIP_FLOOR       = 0.008

In [233]:
points = np.asarray(cleaned_pcd.points)
colors = np.asarray(cleaned_pcd.colors)

In [234]:
farest_distance = np.min(points[:, 2])
lowest_distance = np.min(points[:,1])

# Remove background noise
bg_mask = np.array([points[:, 2] >= farest_distance + (TOOLS_MIN_LENGTH - 0.1 * TOOLS_MIN_LENGTH)])

# Remove foreground noise
fg_mask = np.array([points[:, 2] <= farest_distance + TOOLS_MAX_LENGTH])

# Remove floor noise
fl_mask =  np.array([points[:, 1] >= lowest_distance + CLIP_FLOOR])


filtered_mask = (fg_mask & bg_mask & fl_mask).flatten()

In [235]:
# Filter out points below the Z threshold
filtered_points = points[filtered_mask]
filtered_colors = colors[filtered_mask]

# Create a new point cloud with the filtered points
filtered_point_cloud = o3d.geometry.PointCloud()
filtered_point_cloud.points = o3d.utility.Vector3dVector(filtered_points)
filtered_point_cloud.colors = o3d.utility.Vector3dVector(filtered_colors)

# Visualize the filtered point cloud
o3d.visualization.draw_geometries([filtered_point_cloud])

### Tool recognition

In [242]:
tool_2 = o3d.io.read_triangle_mesh("/calibration/objects/tools/tool_2.stl")
o3d.visualization.draw_geometries([tool_2])

In [ ]:
tool_2.compute_vertex_normals()
pcd_tool = tool_2.sample_points_poisson_disk(3000)
o3d.visualization.draw_geometries([pcd_tool])